In [130]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.externals import joblib
from sklearn.linear_model import PassiveAggressiveClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [260]:
# Data import
rt = pd.read_csv('reviews_rt_all.csv', sep = '|')
imdb = pd.read_csv('imdb_small.csv', sep = '|')

In [261]:
# Split RT and IMDB datasets seapretely
X_train_rt, X_test_rt, y_train_rt, y_test_rt = train_test_split(rt.text, rt.label, test_size=0.2, random_state=42, stratify=rt.label)
X_train_imdb, X_test_imdb, y_train_imdb, y_test_imdb = train_test_split(imdb.text, imdb.label, test_size=0.2, random_state=42, stratify=imdb.label)

# Then concatenate
X_train = pd.concat([X_train_rt, X_train_imdb])
X_test = pd.concat([X_test_rt, X_test_imdb])
y_train = pd.concat([y_train_rt, y_train_imdb])
y_test = pd.concat([y_test_rt, y_test_imdb])

In [262]:
# Take the last 22 words from each review in the train set
X_train = X_train.str.split().apply(lambda x:  ' '.join(x for x in x[-22:]))

In [263]:
# Stopwords
STOPWORDS = ['by','does', 'was', 'were', 'the', 'of', 'end', 'and', 'is']    

In [109]:
vect = CountVectorizer(binary = True,ngram_range=(1,4),stop_words=STOPWORDS)
clf = PassiveAggressiveClassifier(fit_intercept=False,
                                  shuffle=False, n_iter = 91, n_jobs = -1)
pipeline = Pipeline([('vect', vect),
                ('clf', clf)])

parameters = {
#               'tfdf__stop_words': (STOPWORDS, None),
#               'tfdf__binary': (True,False),
#               'tfdf__max_df': (0.5, 0.75, 0,94),
#               'tfdf__min_df': (1,2,3,4),
#               'tfdf__max_features': (25000, 30000, 35000), 
               'clf__C': (0.001, 0.01, 0.1, 1, 10, 100)
              }

In [110]:
%%time

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, error_score=1)
grid_search = grid_search.fit(X_train, y_train)

CPU times: user 46.1 s, sys: 4.33 s, total: 50.4 s
Wall time: 11min 47s


In [111]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, grid_search.best_params_[param_name]))

clf__C: 0.01


In [112]:
n_candidates = len(grid_search.cv_results_['params'])

for i in range(n_candidates):
    print(i, 'params - %s; mean - %0.4f; std - %0.4f'
                 % (grid_search.cv_results_['params'][i],
                    grid_search.cv_results_['mean_test_score'][i],
                    grid_search.cv_results_['std_test_score'][i]))

0 params - {'clf__C': 0.001}; mean - 0.7543; std - 0.0468
1 params - {'clf__C': 0.01}; mean - 0.7547; std - 0.0389
2 params - {'clf__C': 0.1}; mean - 0.7510; std - 0.0385
3 params - {'clf__C': 1}; mean - 0.7439; std - 0.0403
4 params - {'clf__C': 10}; mean - 0.7439; std - 0.0401
5 params - {'clf__C': 100}; mean - 0.7439; std - 0.0401


In [113]:
%%time

# Compare Validation Accuracy on RT, IMDB and mixed test sets
y_pred_rt = grid_search.predict(X_test_rt)
y_pred_imdb = grid_search.predict(X_test_imdb)
y_pred = grid_search.predict(X_test)

print ("Accuracy RT :", metrics.accuracy_score(y_test_rt, y_pred_rt))
print ("Accuracy IMDB :", metrics.accuracy_score(y_test_imdb, y_pred_imdb))
print ("Accuracy RT+IMDB :", metrics.accuracy_score(y_test, y_pred))

Accuracy RT : 0.804551213332
Accuracy IMDB : 0.8975
Accuracy RT+IMDB : 0.835004259223
CPU times: user 25.9 s, sys: 345 ms, total: 26.3 s
Wall time: 26.4 s
